In [614]:
import os
from bs4 import BeautifulSoup
import json
from googletrans import Translator # pip install googletrans==4.0.0-rc1
import fasttext # pip install fastText
import pandas as pd

In [615]:
# 1. get pairs of titles

# Getting the current work directory (cwd)
thisdir = os.getcwd()
html_set = set()
json_set = set()
output_dir_path = '/output_dir/'
# r=root, d=directories, f = files
# 
for r, d, f in os.walk(thisdir+output_dir_path):
    for file in f:
        if file.endswith(".html"):
            html_set.add(file[:-5])
        if file.endswith(".json"):
            json_set.add(file[:-5])


In [616]:
# for file in list(html_set):
#     print('--------------------------------------------')
#     print('file: ',file)
#     path = "output_dir/"+file+".html"
#     html = open(path, "rb") 
#     soup = BeautifulSoup(html, 'html.parser')
#     if (soup.title is None):
#         print(file)
#         print('!!!!!!')
#     if (soup.title is not None):
#         print(soup.title.string) 
    
#     if (soup.html is not None):
#         lang = soup.find('html').get('lang')
#         print('language: ',lang)

In [617]:
# missing_lang = []
# missing_title = []
# for file in list(json_set):
#     print('--------------------------------------------')
#     print('file: ',file)
#     path = "output_dir/"+file+".json"
#     with open(path, 'r') as f:
#         json_file = json.load(f)
    
#     print('title: ',json_file['title'])
#     print('language: ',json_file['meta_lang'])
#     if len(json_file['meta_lang'])<2:
#         missing_lang.append(file)
#     if len(json_file['title'])<3:
#         missing_title.append(file)

In [618]:
print(len(missing_title))
len(missing_lang) # so we need language detection tools

0


0

In [619]:
# # try to access missing title from html but failed
# html_missing_title = []
# for file in missing_title:
# #     print('--------------------------------------------')
# #     print('file: ',file)
#     path = "output_dir/"+file+".html"
#     html = open(path, "rb") 
#     soup = BeautifulSoup(html, 'html.parser')
#     title = soup.title
#     if (title is not None) and len(title)>2:
#         print(title.string) 
#     else:
#         html_missing_title.append(file)
# print(len(html_missing_title))

In [620]:
path_to_pretrained_model = 'lid.176.bin'
ft_model = fasttext.load_model(path_to_pretrained_model)

In [621]:
text = 'Konya İl Emniyet Müdürü Mustafa Aydın\'ın acı günü'
pred = ft_model.predict([text])


In [622]:
def get_title_and_lang_in_dict_in_batch(json_list, start, finish):
    missing_lang = []
    missing_title = []
    
    translator = Translator()
    title_lang_dict = {}
    i = 0
    for file in json_list[start:finish]:
        print('---------------------',i,'-----------------------')
        print('file: ',file)
        path = "output_dir/"+file+".json"
        with open(path, 'r') as f:
            json_file = json.load(f)
        title = str(json_file['title'])
        lang = str(json_file['meta_lang'])
        translated_title = ''
        print('title: ', title)
        print('meta_lang: ',lang)
        
        if title == 'Wyborcza.pl':
            title_lang_dict[str(file)] = {'title':title,'meta_lang':json_file['meta_lang'],'translated_lang':lang, 'translated_title':translated_title ,
                                     "keywords":json_file["keywords"],"meta_keywords":json_file["meta_keywords"], "tags":json_file["tags"],"publish_date":json_file["publish_date"],
                                     "text":json_file["text"]}
            continue
        
        if len(lang)<2 and len(title)>=3:
            missing_lang.append(file)
            if lang == 'en':
                lang = translated.detect(title).lang
            else: 
                translated = translator.translate(title)
                lang = translated.src
                translated_title = translated.text
            print('transaltor lang: ',lang)
        elif len(lang)>=2 and len(title)>=3 and lang != 'en':
             translated_title  = translator.translate(title).text

        if len(title)<3:
            missing_title.append(file)
            title = ''
        elif len(title)>=3 and lang == 'en':
            translated_title = title
        i+=1
        print('transalted title: ',translated_title)
        
        title_lang_dict[str(file)] = {'title':title,'meta_lang':json_file['meta_lang'],'translated_lang':lang, 'translated_title':translated_title ,
                                     "keywords":json_file["keywords"],"meta_keywords":json_file["meta_keywords"], "tags":json_file["tags"],"publish_date":json_file["publish_date"],
                                     "text":json_file["text"]}
    return title_lang_dict


In [623]:
# to avoid handshake operation timeout by Google Translate API with batching
json_list = list(json_set)
json_list_len = len(json_list)

i = 0
batch_start_and_finish = []
batch_size = 500
while i < json_list_len:
    j = i+ batch_size
    batch_start_and_finish.append([i,j])
    i = j

print(batch_start_and_finish)

[[0, 500], [500, 1000], [1000, 1500], [1500, 2000], [2000, 2500], [2500, 3000], [3000, 3500], [3500, 4000], [4000, 4500], [4500, 5000], [5000, 5500], [5500, 6000]]


In [624]:
# we had to run all cells for each batch to avoid the timeout by 
# batch = batch_start_and_finish[i]




# i = 18
# s = 3600
# f = 3800
# i = 19
# s = 3800
# f = 4000
# i = 20
# s = 4000
# f = 4200
# i = 21
# s = 4200
# f = 4400
# i = 22
# s = 4400
# f = 4600
# i = 23
# s = 4600
# f = 4800
# i = 24
# s = 4800
# f = 5000
# i = 25
# s = 5000
# f = 5200
# i = 26
# s = 5200
# f = 5400
i = 27
s = 5400
f = 5600

title_lang_dict = get_title_and_lang_in_dict_in_batch(json_list, s,f)#batch[0], batch[1])


--------------------- 0 -----------------------
file:  1484038028
title:  Syd Mead, visual artist behind Blade Runner, dies at 86
meta_lang:  en
transalted title:  Syd Mead, visual artist behind Blade Runner, dies at 86
--------------------- 1 -----------------------
file:  1502026410
title:  32 killed in attack in disputed South Sudan border region: official
meta_lang:  
transaltor lang:  en
transalted title:  32 killed in attack in disputed South Sudan border region: official
--------------------- 2 -----------------------
file:  1484212827
title:  Redirect Notice
meta_lang:  en
transalted title:  Redirect Notice
--------------------- 3 -----------------------
file:  1483803388
title:  Mindtree Ltd soars 1.3%, rises for third straight session
meta_lang:  en
transalted title:  Mindtree Ltd soars 1.3%, rises for third straight session
--------------------- 4 -----------------------
file:  1483968177
title:  'I've been able to adjust and find a way': Peter Siddle speaks about his cricke

transalted title:  ▷ Unification for gas transfers: Dependence on imports / daily mirror from Finn ...
--------------------- 37 -----------------------
file:  1575834546
title:  VIDEO: Gowdy Doubts Dems Can Persuade a Single GOP Senator on Impeachment Trial
meta_lang:  en
transalted title:  VIDEO: Gowdy Doubts Dems Can Persuade a Single GOP Senator on Impeachment Trial
--------------------- 38 -----------------------
file:  1554362580
title:  Coronavirus may stick around longer than you think — here's what you need to know
meta_lang:  en
transalted title:  Coronavirus may stick around longer than you think — here's what you need to know
--------------------- 39 -----------------------
file:  1484013651
title:  DPRK presents Cuba with flowers on 61st anniversary of Revolution
meta_lang:  en
transalted title:  DPRK presents Cuba with flowers on 61st anniversary of Revolution
--------------------- 40 -----------------------
file:  1484049194
title:  Brände: Verheerender Zoo-Brand: Möglich

transalted title:  Link to Process at Alleged Pedrast
--------------------- 73 -----------------------
file:  1484036974
title:  Beyond Racial Strife, a Dose of Optimism for the New Year
meta_lang:  en
transalted title:  Beyond Racial Strife, a Dose of Optimism for the New Year
--------------------- 74 -----------------------
file:  1484036043
title:  Israel demolishes newly built houses of Palestinian families on New Year's Day –
meta_lang:  en
transalted title:  Israel demolishes newly built houses of Palestinian families on New Year's Day –
--------------------- 75 -----------------------
file:  1510094016
title:  Unsittlicher Angriff In Bremen: Mann bedroht Kinder mit Messer
meta_lang:  de
transalted title:  Immiscal attack in Bremen: Man threatens children with knife
--------------------- 76 -----------------------
file:  1546217024
title:  Some Declines: Ongoing Adirondack Fisher Research -
meta_lang:  en
transalted title:  Some Declines: Ongoing Adirondack Fisher Research -
----

transalted title:  Teacher without training: "Without her, the system would be in front of the collapse"
--------------------- 107 -----------------------
file:  1484011044
title:  Egypt: Uber to acquire Careem, with conditions
meta_lang:  en
transalted title:  Egypt: Uber to acquire Careem, with conditions
--------------------- 108 -----------------------
file:  1544021771
title:  Expoagro tiende puentes para el próximo salto productivo
meta_lang:  es
transalted title:  Expoagro tends bridges for the next productive leap
--------------------- 109 -----------------------
file:  1553953810
title:  ROUNDUP/VIRUS: Ströer zieht Prognosen wegen Corona zurück
meta_lang:  en
transalted title:  ROUNDUP/VIRUS: Ströer zieht Prognosen wegen Corona zurück
--------------------- 110 -----------------------
file:  1510533959
title:  España creció un 2% en 2019: el menor avance desde el inicio de la recuperación
meta_lang:  es
transalted title:  Spain grew by 2% in 2019: the slightest advance since th

transaltor lang:  en
transalted title:  Everything new to Netflix in January and what will no longer be there
--------------------- 142 -----------------------
file:  1484125412
title:  Zagłada ptaków w Polsce. W środę zabijały je wybuchy, w czwartek katami będą...
meta_lang:  
transaltor lang:  pl
transalted title:  Hollow birds in Poland.On Wednesday, the explosions were killed on Thursday, they will be ...
--------------------- 143 -----------------------
file:  1484205799
title:  As Brexit Pulls Britain Apart, It Could Bring Ireland Back Together
meta_lang:  en
transalted title:  As Brexit Pulls Britain Apart, It Could Bring Ireland Back Together


In [625]:
len(title_lang_dict)

145

In [626]:
len(json_set)

5545

In [627]:
df = pd.DataFrame.from_dict(title_lang_dict,orient='index')
df

,title,meta_lang,translated_lang,translated_title,keywords,meta_keywords,tags,publish_date,text
1484038028,"Syd Mead, visual artist behind Blade Runner, d...",en,en,"Syd Mead, visual artist behind Blade Runner, d...",[],[],"[MOVIES, VISUAL ARTS, OBITUARY]",Wed Jan 1 17:52:12 2020,LOS ANGELES (AFP) - American visual artist Syd...
1502026410,32 killed in attack in disputed South Sudan bo...,,en,32 killed in attack in disputed South Sudan bo...,[],"[south sudan, east africa]",[],Wed Jan 22 21:46:39 2020,At least 32 people were killed on Wednesday wh...
1484212827,Redirect Notice,en,en,Redirect Notice,[],[],[],None,
1483803388,"Mindtree Ltd soars 1.3%, rises for third strai...",en,en,"Mindtree Ltd soars 1.3%, rises for third strai...",[],"[BSE SENSEX, NIFTY 50, Economy of India, Nifty...",[Business Finance],None,"Mindtree Ltd is quoting at Rs 810.55, up 1.3% ..."
1483968177,'I've been able to adjust and find a way': Pet...,en,en,'I've been able to adjust and find a way': Pet...,[],"[Peter Siddle retirement, Peter Siddle, Cricke...","[Cricket News, #PeterSiddle, Citizenship Act, ...",Wed Jan 1 17:59:00 2020,Australia speedster Peter Siddle has expressed...
...,...,...,...,...,...,...,...,...,...
1484101809,"AAP slams Congress, previous SAD-BJP alliance ...",en,en,"AAP slams Congress, previous SAD-BJP alliance ...",[],[],[],None,"Chandigarh (Punjab) [India], Jan 1 (ANI): The ..."
1484040153,Start the Year on a Positive Note With These S...,en,en,Start the Year on a Positive Note With These S...,[],[],"[Healthy Living, Self-Care, Books, Netflix]",Wed Jan 1 07:30:13 2020,"Without fail, Netflix has become my winding do..."
1484188201,Everything new to Netflix in January and what ...,,en,Everything new to Netflix in January and what ...,[],[Netflix],[],Wed Jan 1 10:42:52 2020,Sign up to FREE daily email alerts from liverp...
1484125412,Zagłada ptaków w Polsce. W środę zabijały je w...,,pl,"Hollow birds in Poland.On Wednesday, the explo...",[],[],[],Wed Jan 1 19:56:23 2020,Sylwestrowe pokazy sztucznych ogni doprowadził...


In [628]:
os.makedirs('1_training_csv/', exist_ok=True)  
path = '1_training_csv/'+str(i)+'.csv'
df.to_csv(path)  